In [ ]:
import os
import pandas as pd
import json

# Cargar el diccionario desde el archivo JSON
#with open('Project_countries.json', 'r') as json_file:
# country_region_continent = json.load(json_file)

# Leer el archivo JSON con la codificación correcta
with open('Project_countries.json', 'r', encoding='utf-8') as file:
    Project_countries = json.load(file)

# Crear un diccionario inverso para los nombres alternativos
alternate_names_to_official = {}
for country, info in Project_countries.items():
    for alt_name in info.get('AlternateNames', []):
        alternate_names_to_official[alt_name] = country

# Definir la ruta de la carpeta
folder_path = 'Datos/Consolidado 2/Archivos'

# Obtener una lista de todos los archivos CSV en la carpeta
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Inicializar un DataFrame vacío para almacenar todos los datos transpuestos
all_transposed_data = pd.DataFrame()

# Leer y procesar cada archivo CSV
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    try:
        # Leer el archivo CSV con el delimitador adecuado
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        # Encontrar el índice de la fila "Country Name"
        start_index = None
        for i, line in enumerate(lines):
            if line.startswith('"Country Name"'):
                start_index = i
                break
        
        if start_index is None:
            print(f'No se encontró la fila "Country Name" en el archivo {csv_file}')
            continue
                
        # Leer el archivo CSV desde la fila "Country Name"
        data = pd.read_csv(file_path, encoding='utf-8', delimiter=',', quotechar='"', on_bad_lines='skip', skiprows=start_index)
        
        # Reemplazar nombres alternativos por nombres oficiales
        data['Country Name'] = data['Country Name'].apply(lambda x: alternate_names_to_official.get(x, x))  
        
        # Validar que todos los países de la primera columna se encuentren en el archivo JSON
        valid_countries = data['Country Name'].isin(Project_countries.keys())
        data_cleaned = data[valid_countries]
        
        # Eliminar las filas vacías del DataFrame original
        filas_vacias = data_cleaned[data_cleaned.iloc[:, 0].isnull() | (data_cleaned.iloc[:, 0] == '')]
        data_cleaned = data_cleaned.drop(filas_vacias.index)
        
        # Eliminar columnas de años que no se necesitan
        columns_to_keep = list(data_cleaned.columns[:4]) + list(data_cleaned.columns[-10:])
        data_cleaned = data_cleaned[columns_to_keep]

        # Crear columnas adicionales para Región y Continente
        data_cleaned['Region'] = data_cleaned['Country Name'].apply(lambda x: country_region_continent.get(x, {}).get('Region', 'Región Desconocida'))
        data_cleaned['Continent'] = data_cleaned['Country Name'].apply(lambda x: country_region_continent.get(x, {}).get('Continent', 'Continente Desconocido'))
        
        # Crear una lista para almacenar las filas transpuestas
        transposed_data = []
        
        # Se define desde que columna lee los años
        years = data_cleaned.columns[4:]

        # Iterar sobre cada fila del DataFrame original
        for index, row in data_cleaned.iterrows():
            country_name = row['Country Name']
            country_code = row['Country Code']
            indicator_name = row['Indicator Name']
            indicator_code = row['Indicator Code']
            region = row['Region']
            continent = row['Continent']
    
            # Iterar sobre las columnas de años
            for year in years[:-2]:  # Excluir 'Unnamed: 68' y 'Most_Recent_Value'
                value = row[year]
                if not pd.isna(value):
                    transposed_data.append([country_name, country_code, indicator_name, indicator_code, year, value, region, continent])

        # Crear un nuevo DataFrame con los datos transpuestos
        transposed_df = pd.DataFrame(transposed_data, columns=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', 'Year', 'Value', 'Region', 'Continent'])

        # Concatenar los datos transpuestos al DataFrame principal
        all_transposed_data = pd.concat([all_transposed_data, transposed_df], ignore_index=True)
        
    except pd.errors.ParserError as e:
        print(f'Error al procesar el archivo {csv_file}: {e}')

# Ordenar el DataFrame combinado por el nombre de la columna "Country Name"
all_transposed_data.sort_values(by=['Country Name', 'Indicator Name'], inplace=True)

# Mostrar el DataFrame combinado ordenado
print(all_transposed_data.head())

# Guardar el DataFrame combinado en un archivo CSV

# Definir la ruta de la carpeta
New_folder_path = 'Datos'
output_file_path = os.path.join(New_folder_path, 'transposed_data.csv')
all_transposed_data.to_csv(output_file_path, index=False)
print(f'El archivo CSV combinado se ha guardado en: {output_file_path}')

Versión 2

In [ ]:
import os
import pandas as pd

# Definir la ruta de la carpeta
folder_path = 'Datos/Consolidado 2/Archivos'

# Obtener una lista de todos los archivos CSV en la carpeta
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Inicializar un DataFrame vacío para almacenar todos los datos transpuestos
all_transposed_data = pd.DataFrame()

# Leer y procesar cada archivo CSV
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    try:
        # Leer el archivo CSV con el delimitador adecuado
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        # Encontrar el índice de la fila "Country Name"
        start_index = None
        for i, line in enumerate(lines):
            if line.startswith('"Country Name"'):
                start_index = i
                break
        
        if start_index is None:
            print(f'No se encontró la fila "Country Name" en el archivo {csv_file}')
            continue
                
        # Leer el archivo CSV desde la fila "Country Name"
        data = pd.read_csv(file_path, encoding='utf-8', delimiter=',', quotechar='"', on_bad_lines='skip', skiprows=start_index)
        
        data
        
        
        
        filas_vacias = data[data.iloc[:, 0].isnull() | (data.iloc[:, 0] == '')]
        
        #print(f'filas vacias:')
        #print (filas_vacias)
        
        # Eliminar las filas vacías del DataFrame original
        data_cleaned = data.drop(filas_vacias.index)
        
        
        # Eliminar las filas vacías del DataFrame original
        #data_cleaned = data.dropna(subset=[data.columns[0]])
        
        # Se define desde que columna lee los años
        years = data_cleaned.columns[4:]

        # Crear una lista para almacenar las filas transpuestas
        transposed_data = []

        # Iterar sobre cada fila del DataFrame original
        for index, row in data_cleaned.iterrows():
            country_name = row['Country Name']
            country_code = row['Country Code']
            indicator_name = row['Indicator Name']
            indicator_code = row['Indicator Code']
    
            # Iterar sobre las columnas de años
            for year in years[:-2]:  # Excluir 'Unnamed: 68' y 'Most_Recent_Value'
                value = row[year]
                if not pd.isna(value):
                    transposed_data.append([country_name, country_code, indicator_name, indicator_code, year, value])

        # Crear un nuevo DataFrame con los datos transpuestos
        transposed_df = pd.DataFrame(transposed_data, columns=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', 'Year', 'Value'])

        # Concatenar los datos transpuestos al DataFrame principal
        all_transposed_data = pd.concat([all_transposed_data, transposed_df], ignore_index=True)
        
    except pd.errors.ParserError as e:
        print(f'Error al procesar el archivo {csv_file}: {e}')

# Ordenar el DataFrame combinado por el nombre de la columna "Country Name"
all_transposed_data.sort_values(by=['Country Name', 'Indicator Name'], inplace=True)

# Mostrar el DataFrame combinado ordenado
print(all_transposed_data.head())

# Guardar el DataFrame combinado en un archivo CSV
output_file_path = os.path.join(folder_path, 'transposed_data.csv')
all_transposed_data.to_csv(output_file_path, index=False)
print(f'El archivo CSV combinado se ha guardado en: {output_file_path}')

Versiòn 3

In [21]:
import os
import pandas as pd
import json

# Cargar el diccionario desde el archivo JSON
with open('country_region_continent.json', 'r') as json_file:
    country_region_continent = json.load(json_file)

# Definir la ruta de la carpeta
folder_path = 'Datos/Consolidado 2/Archivos'

# Obtener una lista de todos los archivos CSV en la carpeta
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Inicializar un DataFrame vacío para almacenar todos los datos transpuestos
all_transposed_data = pd.DataFrame()

# Leer y procesar cada archivo CSV
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    try:
        # Leer el archivo CSV con el delimitador adecuado
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        # Encontrar el índice de la fila "Country Name"
        start_index = None
        for i, line in enumerate(lines):
            if line.startswith('"Country Name"'):
                start_index = i
                break
        
        if start_index is None:
            print(f'No se encontró la fila "Country Name" en el archivo {csv_file}')
            continue
                
        # Leer el archivo CSV desde la fila "Country Name"
        data = pd.read_csv(file_path, encoding='utf-8', delimiter=',', quotechar='"', on_bad_lines='skip', skiprows=start_index)
               
        # Eliminar las filas vacías del DataFrame original
        filas_vacias = data[data.iloc[:, 0].isnull() | (data.iloc[:, 0] == '')]
        data_cleaned = data.drop(filas_vacias.index)
        
        # Eliminar columnas de años que no se necesitan
        columns_to_keep = list(data_cleaned.columns[:4]) + list(data_cleaned.columns[-10:])
        data_cleaned = data_cleaned[columns_to_keep]

        # Crear columnas adicionales para Región y Continente
        data_cleaned['Region'] = data_cleaned['Country Name'].apply(lambda x: country_region_continent.get(x, {}).get('Region', 'Región Desconocida'))
        data_cleaned['Continent'] = data_cleaned['Country Name'].apply(lambda x: country_region_continent.get(x, {}).get('Continent', 'Continente Desconocido'))
        
        # Crear una lista para almacenar las filas transpuestas
        transposed_data = []
        
        # Se define desde que columna lee los años
        years = data_cleaned.columns[4:]

        # Iterar sobre cada fila del DataFrame original
        for index, row in data_cleaned.iterrows():
            country_name = row['Country Name']
            country_code = row['Country Code']
            indicator_name = row['Indicator Name']
            indicator_code = row['Indicator Code']
            region = row['Region']
            continent = row['Continent']
    
            # Iterar sobre las columnas de años
            for year in years[:-2]:  # Excluir 'Unnamed: 68' y 'Most_Recent_Value'
                value = row[year]
                if not pd.isna(value):
                    transposed_data.append([country_name, country_code, indicator_name, indicator_code, year, value, region, continent])

        # Crear un nuevo DataFrame con los datos transpuestos
        transposed_df = pd.DataFrame(transposed_data, columns=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', 'Year', 'Value', 'Region', 'Continent'])

        # Concatenar los datos transpuestos al DataFrame principal
        all_transposed_data = pd.concat([all_transposed_data, transposed_df], ignore_index=True)
        
    except pd.errors.ParserError as e:
        print(f'Error al procesar el archivo {csv_file}: {e}')

# Ordenar el DataFrame combinado por el nombre de la columna "Country Name"
all_transposed_data.sort_values(by=['Country Name', 'Indicator Name'], inplace=True)

# Mostrar el DataFrame combinado ordenado
print(all_transposed_data.head())

# Guardar el DataFrame combinado en un archivo CSV
output_file_path = os.path.join(folder_path, 'transposed_data.csv')
all_transposed_data.to_csv(output_file_path, index=False)
print(f'El archivo CSV combinado se ha guardado en: {output_file_path}')


unique_countries = all_transposed_data[['Country Name', 'Country Code', 'Region', 'Continent']].drop_duplicates()

# Guardar los valores únicos en un archivo CSV
unique_countries.to_csv('unique_countries.csv', index=False)
print('El archivo CSV con los valores únicos se ha guardado como unique_countries.csv')

No se encontró la fila "Country Name" en el archivo transposed_data.csv
          Country Name Country Code  \
1288   Ingreso mediano          MIC   
1289   Ingreso mediano          MIC   
1290   Ingreso mediano          MIC   
1291   Ingreso mediano          MIC   
1292   Ingreso mediano          MIC   

                                         Indicator Name  Indicator Code  Year  \
1288  Agricultura, silvicultura y pesca, valor agreg...  NV.AGR.TOTL.ZS  2015   
1289  Agricultura, silvicultura y pesca, valor agreg...  NV.AGR.TOTL.ZS  2016   
1290  Agricultura, silvicultura y pesca, valor agreg...  NV.AGR.TOTL.ZS  2017   
1291  Agricultura, silvicultura y pesca, valor agreg...  NV.AGR.TOTL.ZS  2018   
1292  Agricultura, silvicultura y pesca, valor agreg...  NV.AGR.TOTL.ZS  2019   

         Value              Region               Continent  
1288  9.288169  Región Desconocida  Continente Desconocido  
1289  9.274337  Región Desconocida  Continente Desconocido  
1290  8.866479  Región 

In [ ]:
# Obtener los valores únicos de la columna 'Country Name'
unique_countries = all_transposed_data[['Country Name', 'Country Code', 'Region', 'Continent']].drop_duplicates()

# Guardar los valores únicos en un archivo CSV
unique_countries.to_csv('unique_countries.csv', index=False)
print('El archivo CSV con los valores únicos se ha guardado como unique_countries.csv')

Versión 4

In [57]:
import os
import pandas as pd
import json

# Cargar el diccionario desde el archivo JSON
#with open('Project_countries.json', 'r') as json_file:
# country_region_continent = json.load(json_file)

# Leer el archivo JSON con la codificación correcta
with open('Project_countries.json', 'r', encoding='utf-8') as file:
    Project_countries = json.load(file)

# Crear un diccionario inverso para los nombres alternativos
alternate_names_to_official = {}
for country, info in Project_countries.items():
    for alt_name in info.get('AlternateNames', []):
        alternate_names_to_official[alt_name] = country

# Definir la ruta de la carpeta
folder_path = 'Datos/Consolidado 2/Archivos'

# Obtener una lista de todos los archivos CSV en la carpeta
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Inicializar un DataFrame vacío para almacenar todos los datos transpuestos
all_transposed_data = pd.DataFrame()

# Leer y procesar cada archivo CSV
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    try:
        # Leer el archivo CSV con el delimitador adecuado
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        # Encontrar el índice de la fila "Country Name"
        start_index = None
        for i, line in enumerate(lines):
            if line.startswith('"Country Name"'):
                start_index = i
                break
        
        if start_index is None:
            print(f'No se encontró la fila "Country Name" en el archivo {csv_file}')
            continue
                
        # Leer el archivo CSV desde la fila "Country Name"
        data = pd.read_csv(file_path, encoding='utf-8', delimiter=',', quotechar='"', on_bad_lines='skip', skiprows=start_index)
        
        # Reemplazar nombres alternativos por nombres oficiales
        data['Country Name'] = data['Country Name'].apply(lambda x: alternate_names_to_official.get(x, x))  
        
        # Validar que todos los países de la primera columna se encuentren en el archivo JSON
        valid_countries = data['Country Name'].isin(Project_countries.keys())
        data_cleaned = data[valid_countries]
        
        # Eliminar las filas vacías del DataFrame original
        filas_vacias = data_cleaned[data_cleaned.iloc[:, 0].isnull() | (data_cleaned.iloc[:, 0] == '')]
        data_cleaned = data_cleaned.drop(filas_vacias.index)
        
        # Eliminar columnas de años que no se necesitan
        columns_to_keep = list(data_cleaned.columns[:4]) + list(data_cleaned.columns[-10:])
        data_cleaned = data_cleaned[columns_to_keep]

        # Crear columnas adicionales para Región y Continente
        data_cleaned['Region'] = data_cleaned['Country Name'].apply(lambda x: country_region_continent.get(x, {}).get('Region', 'Región Desconocida'))
        data_cleaned['Continent'] = data_cleaned['Country Name'].apply(lambda x: country_region_continent.get(x, {}).get('Continent', 'Continente Desconocido'))
        
        # Crear una lista para almacenar las filas transpuestas
        transposed_data = []
        
        # Se define desde que columna lee los años
        years = data_cleaned.columns[4:]

        # Iterar sobre cada fila del DataFrame original
        for index, row in data_cleaned.iterrows():
            country_name = row['Country Name']
            country_code = row['Country Code']
            indicator_name = row['Indicator Name']
            indicator_code = row['Indicator Code']
            region = row['Region']
            continent = row['Continent']
    
            # Iterar sobre las columnas de años
            for year in years[:-2]:  # Excluir 'Unnamed: 68' y 'Most_Recent_Value'
                value = row[year]
                if not pd.isna(value):
                    transposed_data.append([country_name, country_code, indicator_name, indicator_code, year, value, region, continent])

        # Crear un nuevo DataFrame con los datos transpuestos
        transposed_df = pd.DataFrame(transposed_data, columns=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', 'Year', 'Value', 'Region', 'Continent'])

        # Concatenar los datos transpuestos al DataFrame principal
        all_transposed_data = pd.concat([all_transposed_data, transposed_df], ignore_index=True)
        
    except pd.errors.ParserError as e:
        print(f'Error al procesar el archivo {csv_file}: {e}')

# Ordenar el DataFrame combinado por el nombre de la columna "Country Name"
all_transposed_data.sort_values(by=['Country Name', 'Indicator Name'], inplace=True)

# Mostrar el DataFrame combinado ordenado
print(all_transposed_data.head())

# Guardar el DataFrame combinado en un archivo CSV

# Definir la ruta de la carpeta
New_folder_path = 'Datos'
output_file_path = os.path.join(New_folder_path, 'transposed_data.csv')
all_transposed_data.to_csv(output_file_path, index=False)
print(f'El archivo CSV combinado se ha guardado en: {output_file_path}')

No se encontró la fila "Country Name" en el archivo transposed_data.csv
  Country Name Country Code  \
5   Afganistán          AFG   
6   Afganistán          AFG   
7   Afganistán          AFG   
8   Afganistán          AFG   
9   Afganistán          AFG   

                                      Indicator Name  Indicator Code  Year  \
5  Agricultura, silvicultura y pesca, valor agreg...  NV.AGR.TOTL.ZS  2015   
6  Agricultura, silvicultura y pesca, valor agreg...  NV.AGR.TOTL.ZS  2016   
7  Agricultura, silvicultura y pesca, valor agreg...  NV.AGR.TOTL.ZS  2017   
8  Agricultura, silvicultura y pesca, valor agreg...  NV.AGR.TOTL.ZS  2018   
9  Agricultura, silvicultura y pesca, valor agreg...  NV.AGR.TOTL.ZS  2019   

       Value              Region               Continent  
5  20.634323  Región Desconocida  Continente Desconocido  
6  25.740314  Región Desconocida  Continente Desconocido  
7  26.420199  Región Desconocida  Continente Desconocido  
8  22.042897  Región Desconocida  Co

Version 5 leer archivos happiness

In [61]:
import os
import pandas as pd
import json

# Define la ruta de la carpeta y el archivo JSON
folder_path = 'Datos/Happiness'
#json_path = 'Project_countries.json'

# Define los mapeos de columnas
column_mappings = {
    'Country': ['Country', 'Country or region','Country name'],
    'Rank': ['Happiness Rank', 'Overall rank', 'Happiness.Rank','Rank'],
    'Score': ['Happiness Score', 'Score', 'Happiness.Score','Ladder score','Happiness score'],
    'GDP per capita': ['Economy (GDP per Capita)', 'Economy..GDP.per.Capita.', 'GDP per capita','Logged GDP per capita','Explained by: GDP per capita'],
    'Family': ['Family', 'Social support','Explained by: Social support'],
    'Health (Life Expectancy)': ['Health (Life Expectancy)', 'Health..Life.Expectancy.', 'Healthy life expectancy','Explained by: Healthy life expectancy'],
    'Freedom': ['Freedom', 'Freedom to make life choices','Explained by: Freedom to make life choices'],
    'Perceptions of corruption': ['Trust (Government Corruption)', 'Trust..Government.Corruption.', 'Perceptions of corruption','Explained by: Perceptions of corruption'],
    'Generosity': ['Generosity','Explained by: Generosity']
}

# Crear un diccionario inverso para los nombres alternativos
alternate_names_to_official = {}
for country, info in Project_countries.items():
    for alt_name in info.get('AlternateNames', []):
        alternate_names_to_official[alt_name] = country

# Función para estandarizar los nombres de las columnas
def standardize_columns(df):
    standardized_columns = {}
    for standard_col, possible_cols in column_mappings.items():
        for col in possible_cols:
            if col in df.columns:
                standardized_columns[col] = standard_col
                break
    return df.rename(columns=standardized_columns)

# Leer el archivo JSON con la codificación correcta
with open('Project_countries.json', 'r', encoding='utf-8') as file:
    project_countries = json.load(file)

# Lista para almacenar los dataframes
dataframes = []

# Iterar sobre los archivos en la carpeta
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        # Extraer el año del nombre del archivo
        year = int(''.join(filter(str.isdigit, filename)))
        
        # Leer el archivo CSV
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        
        # Estandarizar las columnas
        df = standardize_columns(df)
        
        # Asignar Rank si no existe en el archivo
        if 'Rank' not in df.columns:
            df['Rank'] = range(1, len(df) + 1)
        
        # Reemplazar nombres alternativos por nombres oficiales
        df['Country'] = df['Country'].apply(lambda x: alternate_names_to_official.get(x, x))
        
        # Filtrar las filas basadas en el archivo JSON
        df = df[df['Country'].isin(project_countries)]
        
        # Filtrar las columnas relevantes y agregar la columna de año
        df = df[['Country', 'Rank', 'Score', 'GDP per capita', 'Family', 
                 'Health (Life Expectancy)', 'Freedom', 
                 'Perceptions of corruption', 'Generosity']]
        df['Year'] = year
        
        # Formatear los datos numéricos para que solo tengan 3 dígitos
        numeric_columns = ['Rank', 'Score', 'GDP per capita', 'Family', 
                           'Health (Life Expectancy)', 'Freedom', 
                           'Perceptions of corruption', 'Generosity']
        df[numeric_columns] = df[numeric_columns].applymap(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)
        
        
        # Formatear los datos numéricos para que solo tengan 3 dígitos
        #numeric_columns = ['Rank', 'Score', 'GDP per capita', 'Family', 
        #                   'Health (Life Expectancy)', 'Freedom', 
        #                   'Perceptions of corruption', 'Generosity']
        #df[numeric_columns] = df[numeric_columns].apply(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)
        
        
        
        # Agregar el dataframe a la lista
        dataframes.append(df)

# Concatenar todos los dataframes
consolidated_df = pd.concat(dataframes)

# Ordenar el DataFrame combinado por el nombre de la columna "Country Name"
consolidated_df.sort_values(by=['Country'], inplace=True)

# Guardar el dataframe consolidado en un nuevo archivo CSV
consolidated_df.to_csv('consolidado_happiness.csv', index=False)

print("Consolidación completa. Los datos consolidados se han guardado en consolidado_happiness.csv.")

Consolidación completa. Los datos consolidados se han guardado en consolidado_happiness.csv.


Version 6

In [84]:
import os
import pandas as pd
import json

# Define la ruta de la carpeta y el archivo JSON
folder_path = 'Datos/Happiness'
json_path = 'Project_countries.json'

# Define los mapeos de columnas
column_mappings = {
    'Country': ['Country', 'Country or region','Country name'],
    'Rank': ['Happiness Rank', 'Overall rank', 'Happiness.Rank','Rank'],
    'Score': ['Happiness Score', 'Score', 'Happiness.Score','Ladder score','Happiness score'],
    'GDP per capita': ['Economy (GDP per Capita)', 'Economy..GDP.per.Capita.', 'GDP per capita','Logged GDP per capita','Explained by: GDP per capita'],
    'Family': ['Family', 'Social support','Explained by: Social support'],
    'Health (Life Expectancy)': ['Health (Life Expectancy)', 'Health..Life.Expectancy.', 'Healthy life expectancy','Explained by: Healthy life expectancy'],
    'Freedom': ['Freedom', 'Freedom to make life choices','Explained by: Freedom to make life choices'],
    'Perceptions of corruption': ['Trust (Government Corruption)', 'Trust..Government.Corruption.', 'Perceptions of corruption','Explained by: Perceptions of corruption'],
    'Generosity': ['Generosity','Explained by: Generosity']
}

# Crear un diccionario inverso para los nombres alternativos
alternate_names_to_official = {}
with open(json_path, 'r', encoding='utf-8') as file:
    project_countries = json.load(file)
    for country, info in project_countries.items():
        for alt_name in info.get('AlternateNames', []):
            alternate_names_to_official[alt_name] = country

# Función para estandarizar los nombres de las columnas y eliminar las no deseadas
def standardize_columns(df):
    standardized_columns = {}
    for standard_col, possible_cols in column_mappings.items():
        for col in possible_cols:
            if col in df.columns:
                standardized_columns[col] = standard_col
                break
    df = df.rename(columns=standardized_columns)
    # Eliminar columnas que no están en los mapeos
    columns_to_keep = list(standardized_columns.values())
    df = df[columns_to_keep]
    return df

# Lista para almacenar los dataframes
dataframes = []

# Iterar sobre los archivos en la carpeta
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        # Extraer el año del nombre del archivo
        year = int(''.join(filter(str.isdigit, filename)))
        
        # Leer el archivo CSV
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        
        # Eliminar filas vacías
        df.dropna(how='all', inplace=True)
        
        # Estandarizar las columnas y eliminar las no deseadas
        df = standardize_columns(df)
        
        # Asignar Rank si no existe en el archivo
        if 'Rank' not in df.columns:
            df['Rank'] = range(1, len(df) + 1)
        
        # Asegurarse de que los valores de 'Country' sean cadenas y manejar valores nulos
        df['Country'] = df['Country'].astype(str).fillna('')
        
        # Reemplazar nombres alternativos por nombres oficiales
        df['Country'] = df['Country'].apply(lambda x: alternate_names_to_official.get(x, x))
        
        # Filtrar las filas basadas en el archivo JSON
        df = df[df['Country'].isin(project_countries)]
        
        # Redondear las columnas numéricas a 3 dígitos si existen
        numeric_columns = ['Rank', 'Score', 'GDP per capita', 'Family', 
                           'Health (Life Expectancy)', 'Freedom', 
                           'Perceptions of corruption', 'Generosity']
        for col in numeric_columns:
            if col in df.columns:
                df[col] = df[col].apply(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)
        
        # Renombrar las columnas para incluir el año
        df = df.rename(columns={col: f"{col} ({year})" for col in df.columns if col != 'Country'})
        
        # Agregar el dataframe a la lista
        dataframes.append(df)

# Concatenar todos los dataframes por columnas
consolidated_df = pd.concat(dataframes, axis=1)

# Eliminar columnas duplicadas de 'Country'
consolidated_df = consolidated_df.loc[:,~consolidated_df.columns.duplicated()]

# Redondear las columnas numéricas a 3 dígitos en el dataframe consolidado si existen
for col in consolidated_df.columns:
    if any(numeric_col in col for numeric_col in numeric_columns):
        consolidated_df[col] = consolidated_df[col].apply(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)

# Ordenar el DataFrame combinado por el nombre de la columna "Country"
consolidated_df.sort_values(by=['Country'], inplace=True)

# Guardar el dataframe consolidado en un nuevo archivo CSV
consolidated_df.to_csv('consolidado_happiness_v2.csv', index=False)

print("Consolidación completa. Los datos consolidados se han guardado en consolidado_happiness_v2.csv.")

Consolidación completa. Los datos consolidados se han guardado en consolidado_happiness_v2.csv.


In [88]:
import os
import pandas as pd
import json

# Define la ruta de la carpeta y el archivo JSON
folder_path = 'Datos/Happiness'
json_path = 'Project_countries.json'

# Define los mapeos de columnas
column_mappings = {
    'Country': ['Country', 'Country or region','Country name'],
    'Rank': ['Happiness Rank', 'Overall rank', 'Happiness.Rank','Rank'],
    'Score': ['Happiness Score', 'Score', 'Happiness.Score','Ladder score','Happiness score'],
    'GDP per capita': ['Economy (GDP per Capita)', 'Economy..GDP.per.Capita.', 'GDP per capita','Logged GDP per capita','Explained by: GDP per capita'],
    'Family': ['Family', 'Social support','Explained by: Social support'],
    'Health (Life Expectancy)': ['Health (Life Expectancy)', 'Health..Life.Expectancy.', 'Healthy life expectancy','Explained by: Healthy life expectancy'],
    'Freedom': ['Freedom', 'Freedom to make life choices','Explained by: Freedom to make life choices'],
    'Perceptions of corruption': ['Trust (Government Corruption)', 'Trust..Government.Corruption.', 'Perceptions of corruption','Explained by: Perceptions of corruption'],
    'Generosity': ['Generosity','Explained by: Generosity']
}

# Crear un diccionario inverso para los nombres alternativos
alternate_names_to_official = {}
with open(json_path, 'r', encoding='utf-8') as file:
    project_countries = json.load(file)
    for country, info in project_countries.items():
        for alt_name in info.get('AlternateNames', []):
            alternate_names_to_official[alt_name] = country

# Función para estandarizar los nombres de las columnas y eliminar las no deseadas
def standardize_columns(df):
    standardized_columns = {}
    for standard_col, possible_cols in column_mappings.items():
        for col in possible_cols:
            if col in df.columns:
                standardized_columns[col] = standard_col
                break
    df = df.rename(columns=standardized_columns)
    # Eliminar columnas que no están en los mapeos
    columns_to_keep = list(standardized_columns.values())
    df = df[columns_to_keep]
    return df

# Lista para almacenar los dataframes
dataframes = []

# DataFrame consolidado inicial
consolidated_df = pd.DataFrame()

# Iterar sobre los archivos en la carpeta
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        # Extraer el año del nombre del archivo
        year = int(''.join(filter(str.isdigit, filename)))
        
        # Leer el archivo CSV
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        
        # Eliminar filas vacías
        df.dropna(how='all', inplace=True)
        
        # Estandarizar las columnas y eliminar las no deseadas
        df = standardize_columns(df)
        
        # Asegurarse de que la columna Rank sea un entero
        if 'Rank' in df.columns:
            df['Rank'] = df['Rank'].astype(int)
        else:
            df['Rank'] = range(1, len(df) + 1)
        
        # Asegurarse de que los valores de 'Country' sean cadenas y manejar valores nulos
        df['Country'] = df['Country'].astype(str).fillna('')
        
        # Reemplazar nombres alternativos por nombres oficiales
        df['Country'] = df['Country'].apply(lambda x: alternate_names_to_official.get(x, x))
        
        # Filtrar las filas basadas en el archivo JSON
        df = df[df['Country'].isin(project_countries)]
        
        # Redondear las columnas numéricas a 3 dígitos si existen
        numeric_columns = ['Rank', 'Score', 'GDP per capita', 'Family', 
                           'Health (Life Expectancy)', 'Freedom', 
                           'Perceptions of corruption', 'Generosity']
        for col in numeric_columns:
            if col in df.columns:
                df[col] = df[col].apply(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)
        
        # Renombrar las columnas para incluir el año
        df = df.rename(columns={col: f"{col} ({year})" for col in df.columns if col != 'Country'})
        
        # Si el DataFrame consolidado está vacío, inicializarlo con el primer DataFrame
        if consolidated_df.empty:
            consolidated_df = df.set_index('Country')
        else:
            # Alinear los datos del nuevo DataFrame con el DataFrame consolidado
            df = df.set_index('Country')
            consolidated_df = consolidated_df.join(df, how='outer')
        
# Ordenar el DataFrame combinado por el nombre de la columna "Country"
consolidated_df.sort_values(by=['Country'], inplace=True)

# Guardar el dataframe consolidado en un nuevo archivo CSV
consolidated_df.to_csv('consolidado_happiness_v2.csv')

print("Consolidación completa. Los datos consolidados se han guardado en consolidado_happiness_v2.csv.")

Consolidación completa. Los datos consolidados se han guardado en consolidado_happiness_v2.csv.


In [92]:
import os
import pandas as pd
import json

# Define la ruta de la carpeta y el archivo JSON
folder_path = 'Datos/Happiness'
json_path = 'Project_countries.json'

# Define los mapeos de columnas
column_mappings = {
    'Country': ['Country', 'Country or region','Country name'],
    'Rank': ['Happiness Rank', 'Overall rank', 'Happiness.Rank','Rank'],
    'Score': ['Happiness Score', 'Score', 'Happiness.Score','Ladder score','Happiness score'],
    'GDP per capita': ['Economy (GDP per Capita)', 'Economy..GDP.per.Capita.', 'GDP per capita','Logged GDP per capita','Explained by: GDP per capita'],
    'Family': ['Family', 'Social support','Explained by: Social support'],
    'Health (Life Expectancy)': ['Health (Life Expectancy)', 'Health..Life.Expectancy.', 'Healthy life expectancy','Explained by: Healthy life expectancy'],
    'Freedom': ['Freedom', 'Freedom to make life choices','Explained by: Freedom to make life choices'],
    'Perceptions of corruption': ['Trust (Government Corruption)', 'Trust..Government.Corruption.', 'Perceptions of corruption','Explained by: Perceptions of corruption'],
    'Generosity': ['Generosity','Explained by: Generosity']
}

# Crear un diccionario inverso para los nombres alternativos
alternate_names_to_official = {}
with open(json_path, 'r', encoding='utf-8') as file:
    project_countries = json.load(file)
    for country, info in project_countries.items():
        for alt_name in info.get('AlternateNames', []):
            alternate_names_to_official[alt_name] = country

# Función para estandarizar los nombres de las columnas y eliminar las no deseadas
def standardize_columns(df):
    standardized_columns = {}
    for standard_col, possible_cols in column_mappings.items():
        for col in possible_cols:
            if col in df.columns:
                standardized_columns[col] = standard_col
                break
    df = df.rename(columns=standardized_columns)
    # Eliminar columnas que no están en los mapeos
    columns_to_keep = list(standardized_columns.values())
    df = df[columns_to_keep]
    return df

# Lista para almacenar los dataframes
dataframes = []

# DataFrame consolidado inicial
consolidated_df = pd.DataFrame()

# Iterar sobre los archivos en la carpeta
for filename in os.listdir(folder_path):
    if filename.startswith('Happiness') and filename.endswith('.csv'):
        # Extraer el año del nombre del archivo
        year = int(''.join(filter(str.isdigit, filename)))
        
        # Leer el archivo CSV
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        
        # Eliminar filas vacías
        df.dropna(how='all', inplace=True)
        
        # Estandarizar las columnas y eliminar las no deseadas
        df = standardize_columns(df)
        
        # Asegurarse de que la columna Rank sea un entero
        if 'Rank' in df.columns:
            df['Rank'] = df['Rank'].astype(int)
        else:
            df['Rank'] = range(1, len(df) + 1)
        
        # Asegurarse de que los valores de 'Country' sean cadenas y manejar valores nulos
        df['Country'] = df['Country'].astype(str).fillna('')
        
        # Reemplazar nombres alternativos por nombres oficiales
        df['Country'] = df['Country'].apply(lambda x: alternate_names_to_official.get(x, x))
        
        # Filtrar las filas basadas en el archivo JSON
        df = df[df['Country'].isin(project_countries)]
        
        # Redondear las columnas numéricas a 3 dígitos si existen
        numeric_columns = ['Rank', 'Score', 'GDP per capita', 'Family', 
                           'Health (Life Expectancy)', 'Freedom', 
                           'Perceptions of corruption', 'Generosity']
        for col in numeric_columns:
            if col in df.columns:
                df[col] = df[col].apply(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)
        
        # Renombrar las columnas para incluir el año
        df = df.rename(columns={col: f"{col} ({year})" for col in df.columns if col != 'Country'})
        
        # Si el DataFrame consolidado está vacío, inicializarlo con el primer DataFrame
        if consolidated_df.empty:
            consolidated_df = df.set_index('Country')
        else:
            # Alinear los datos del nuevo DataFrame con el DataFrame consolidado
            df = df.set_index('Country')
            consolidated_df = consolidated_df.join(df, how='outer')
        
# Ordenar el DataFrame combinado por el nombre de la columna "Country"
consolidated_df.sort_values(by=['Country'], inplace=True)

# Guardar el dataframe consolidado en un nuevo archivo CSV
consolidated_df.to_csv('consolidado_happiness_v2.csv')

print("Consolidación completa. Los datos consolidados se han guardado en consolidado_happiness_v2.csv.")

Consolidación completa. Los datos consolidados se han guardado en consolidado_happiness_v2.csv.


In [93]:
# Verificar el directorio actual de trabajo
current_directory = os.getcwd()
print(f"Directorio actual: {current_directory}")

Directorio actual: c:\Users\ProjectSupport\Documents\David\Cursos\Master Data Science\Proyecto
